<a href="https://colab.research.google.com/github/abs-git/GNN/blob/main/Molecular_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 분자를 molecular graph 형태로 변환하고, 분자의 logP값을 알려주는 패키지 설치

!curl -LO  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

!conda install -y -c rdkit rdkit 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 72.1M  100 72.1M    0     0   174M      0 --:--:-- --:--:-- --:--:--  174M
PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
Solving environment: | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | /

In [ ]:
# 분자를 텍스트 형태로 표현한 smiles 파일과 molecular graph를 생성하는데 필요한 라이브러리

!curl -o ZINC.smiles https://raw.githubusercontent.com/heartcored98/Standalone-DeepLearning/master/Lec9/ZINC.smiles
!curl -o vocab.npy https://raw.githubusercontent.com/heartcored98/Standalone-DeepLearning/master/Lec9/vocab.npy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5374k  100 5374k    0     0  7454k      0 --:--:-- --:--:-- --:--:-- 7454k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256  100   256    0     0   1333      0 --:--:-- --:--:-- --:--:--  1333


In [ ]:
import argparse
import sys
import time
import copy

import numpy as np

# from rdkit import Chem, DataStructues
# from rdkit.Chem import AllChem
# from rdkit.Chem.Crippen import MolLogP 

from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
paser = argparse.ArgumentParser()
args = paser.parse_args("")
args.seed = 123
args.val_size = 0.1
args.test_size = 0.1
args.shuffle = True

In [ ]:
# torch setting

np.random.seed(args.seed)
torch.manual_seed(args.seed)

if torch.cuda.is_available():
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
  torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
# Custom torch dataset
# 두 개의 input (list_feature, list_adj)로 부터 하나의 output (logP Value) 을 얻고자 한다.

class GCNDataset(Dataset):
  def __init__(self, list_feature, list_adj, list_logP):
    self.list_feature = list_feature
    self.list_adj = list_adj
    self.list_logP = list_logP

  def __len__(self):                # 전체 데이터의 개수
    return len(self.list_feature)

  def __getitem__(self, index):             # 특정 위치의 값 
    return self.list_feature[index], self.list_adj[index], self.list_logP[index]


def partition(list_feature, list_adj, list_logP):
  # train, validation, test로 나누는 함수

  num_total = list_feature.shape[0]
  num_train = int(num_total * (1 - args.test_size - args.val_size))   # 0.8
  num_val = int(num_total * args.val_size)
  num_test = int(num_total * args.test_size)

  feature_train = list_feature[ : num_train]
  feature_val = list_feature[num_train : num_train + num_val]
  feature_test = list_feature[num_total - num_test : ]

  adj_train = list_adj[ : num_train]
  adj_val = list_adj[num_train : num_train + num_val]
  adj_test = list_adj[num_total - num_test : ]

  logP_train = list_logP[ : num_train]
  logP_val = list_logP[num_train : num_train + num_val]
  logP_test = list_logP[num_total - num_test : ]
  
  train_set = GCNDataset(feature_train, adj_train, logP_train)
  val_set = GCNDataset(feature_val, adj_val, logP_val)
  test_set = GCNDataset(feature_test, adj_test, logP_test)

  partition  = {
      'train' : train_set,
      'val' : val_set,
      'test' : test_set
  }

In [ ]:
partition = partition(list_feature, list_adj, list_logP, args)

GCN 

adjacent matrix (인접 행렬) * (Hidden state * Weights) → Convolution layer와 같은 기능을 함

adjacent matrix * Node feature matrix ($H^l$)

In [ ]:
# + Attention module

class Attention(nn.Module):

  def __init__(self, in_dim, out_dim, num_head):   # num_head = multi head attention에서 H를 num_head 만큼 서로 다른 weights와 곱한다.
    super(Attention, self).__init__()

    self.num_head = num_head
    self.atn_dim = out_dim // num_head

    self.linears = nn.ModuleList()
    self.corelations - nn.ParameterList()

    for i in range(self.num_head):
      self.linears.append(nn.Linear(in_dim, self.atn_dim))
      corelation = torch.FloatTensor(self.atn_dim, self.atn_dim)
      self.corlations.append(nn.Parameter(corelation))

    self.tanh = torch.nn.Tanh()

  def forward(self, x, adj):
    
    heads = []
    for i in range(self.num_head):
      x_transformed = self.linears[i](x)
      alpha = self.attention_matrix(x_transformed, self.corelation[i], adj)
      x_head = torch.matmul(alpha, x_transformed)
      heads.append(x_head)

    output = torch.cat(heads, dim = 2)

    return output


  def attention_matrix(self, x_transformed, corelation, adj):
    x = torch.einsum('akj, ij -> aki', (x_transformed, corelation))
    alpha = torch.matmul(x, torch.transpose(x_transformed, 1, 2))
    alpha = torch.mul(alpha, adj)
    alpha = self.tanh()

    return alpha


In [ ]:
# Modules

class GCNLayer(nn.Module):          # node feature matrix와 adjacency matrix를 받아 graph convolution 연산을 수행
  
  def __init__(self, in_dim, out_dim, n_atom, act = None, bn = False):
    super(GCNLayer, self).__init__()

    self.use_bn = bn
    self.linear = nn.Linear(in_dim, out_dim)          # convolution의 역할을 함
    nn.init.xavier_uniform_(self.linear.weight)
    self.bn = nn.BatchNorm1d(n_atom)
    self.activation = act

  def forward(self, x, adj):        # 두 개의 input, return 값도 두 개
    out = self.linear(x)            # H * W + bias
    out = torch.matmul(adj, out)    # adj * (H * W + bias)  내적
    
    if self.use_bn:
      out = self.bn(out)
    if self.activation != None:
      out = self.activation(out)
    
    return out, adj


class GCNLayer_with_attention(nn.Module):     
  
  def __init__(self, in_dim, out_dim, n_atom, act = None, bn = False, atn = False, num_head = 1):
    super(GCNLayer_with_attention, self).__init__()

    self.use_bn = bn
    self.linear = nn.Linear(in_dim, out_dim)          # convolution의 역할을 함
    nn.init.xavier_uniform_(self.linear.weight)
    self.bn = nn.BatchNorm1d(n_atom)
    self.activation = act

    self.use_atn = atn
    self.attention = Attention(out_dim, out_dim, num_head)


  def forward(self, x, adj):        # 두 개의 input, return 값도 두 개
    out = self.linear(x)            # H * W + bias

    if self.use_atn:
      out = self.attention(out, adj)
    else :
      out = torch.matmul(adj, out)    # adj * (H * W + bias)  내적
    
    if self.use_bn:
      out = self.bn(out)
    if self.activation != None:
      out = self.activation(out)
    
    return out, adj



In [ ]:
 
class SkipConnection(nn.Module):              # skip connection module

  def __init__(self, in_dim, out_dim):
    super(SkipConnection, self).__init__()

    self.in_dim = in_dim
    self.out_dim = out_dim

    self.linear = nn.Linear(in_dim, out_dim, bias = False)

  def forward(self, in_x, out_x):
    if (self.in_dim != self.out_dim):
      in_x = self.linear(in_x)
    
    out = in_x + out_x
    return out



class GatedSkipConnection(nn.Module):

  def __init__(self, in_dim, out_dim):
    super(GatedSkipConnection, self).__init__()

    self.in_dim = in_dim
    self.out_dim = out_dim

    self.linear = nn.Linear(in_dim, out_dim, bias = False)
    
    self.linear_coef_in = nn.Linear(out_dim, out_dim)
    self.linear_coef_out = nn.Linear(out_dim, out_dim)
    self.sigmoid = nn.Sigmoid()

  def forward(self, in_x, out_x):
    if (self.in_dim != self.out_dim):
      in_x = self.linear(in_x)

    z = self.gate_coefficient(in_x, out_x)
    out = torch.mul(z, out_x) + torch.mul(1.0 -z, in_x)

    return out

  def gate_coefficient(self, in_x, out_x):
    x1 = self.linear_coef_in(in_x)
    x2 = self.linear_coef_out(out_x)

    return self.sigmoid(x1 + x2)




In [ ]:

class GCNBlock(nn.Module):            # GCN layers와 skip or gated skip connection 를 조합한 블럭
  
  def __init__(self, n_layer, in_dim, hidden_dim, out_dim, n_atom, bn = True, sc = 'gsc', act = None):
    super(GCNBlock, self).__init__()

    self.layers = nn.ModuleList()
    for i in range(n_layer):
      self.layers.append(GCNLayer(in_dim if i == 0 else hidden_dim,                 # 첫째, 마지막 layer 빼곤 hidden layer
                                  out_dim if i == n_layer - 1 else hidden_dim,
                                  n_atom,
                                  act if i != n_layer - 1 else None,
                                  bn
                                  ))

      self.relu = nn.ReLU()
      if sc == 'gsc':
        self.sc = GatedSkipConnection(in_dim, out_dim)
      elif sc == 'sc' :
        self.sc = SkipConnection(in_dim, out_dim)
      elif sc == 'no':
        self.sc = None
      else:
        assert False, "Check the skip connection type."

    self.activation = act
  
  def forward(self, x, adj):
    identity = x

    for i, layer in enumerate(self.layers):
      out, adj = layer((x if i == 0 else out), adj)
    
    if self.sc != None:
      out = self.sc(identity, out)

    if self.activation != None:
      out = self.activation(out)

    return out, adj



In [ ]:
# 같은 그래프지만 노드에 부여된 번호에 따라 행렬이 변화가 되는 문제를 해결하기 위해 Readout layer를 거친다.


class ReadOut(nn.Module):             # graph structrure에 permutation invariance를 주기 위하여 linear layer를 거친 뒤 batch 별로 summation하는 module
                                      # MLP 통과 후 합치는 것과 같음
  def __init__(self, in_dim, out_dim, act = None):
    super(ReadOut, self).__init__()

    self.in_dim = in_dim
    self.out_dim = out_dim

    self.linear = nn.Linear(self.in_dim,
                            self.out_dim)
    nn.init.xavier_uniform_(self.linear.weight)
    self.activation = act

    def forward(self, x):
      out = self.linear(x)
      out = torch.sum(out, 1)

      if self.activation != None:
        out = self.activation(out)
      
      return out


class Predictor(nn.Module):

  def __init__(self, in_dim, out_dim, act = None):
    super(Predictor, self).__init__()

    self.in_dim = in_dim
    self.out_dim = out_dim

    self.linear = nn.Linear(self.in_dim,
                            self.out_dim)
    nn.init.xavier_uniform_(self.linear.weight)
    self.activation = act

  def forward(self, x):
    out = self.linear(x)

    if self.activation != None:
      out = self.activation(out)

    return out


In [ ]:
# Model

class GCNNet(nn.Module):

  def __init__(self, args):
    super(GCNNet, self).__init__()

    self.blocks = nn.ModuleList()
    for i in range(args.n_block):
      self.blocks.append(GCNBlock(args.n_layer,
                                  args.in_dim if i == 0 else args.hidden_dim,
                                  args.hidden_dim,
                                  args.hidden_dim,
                                  args.n_atom,
                                  args.bn,
                                  args.sc,
                                  args.act                                  
                                  ))
      self.readout = ReadOut(args.hidden_dim,
                             args.pred_dim1,
                             act = nn.ReLU()
                             )
      self.pred1 = Predictor(args.pred_dim1,
                             args.pred_dim2,
                             act = nn.ReLU()
                             )
      self.pred2 = Predictor(args.pred_dim2,
                             args.pred_dim3,
                             act = nn.Tanh()
                             )
      self.pred3 = Predictor(args.pred_dim3,
                             args.out_dim
                             )

  def forward(self, x, adj):
    for i, block in enumerate(self.blocks):
      out, adj = block((x if i == 0 else out), adj)
    
    out = self.readout(out)
    out = self.pred1(out)
    out = self.pred2(out)
    out = self.pred3(out)

    return out


In [ ]:
# train, validation, test and experiment

def train(net, partition, optimizer, criterion, args):
  train_loader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size = args.train_batch_size,
                                             shuffle = True,
                                             num_workers = 2
                                             )
  net.train()
  optimizer.zero.grad()

  train_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    list_feature, list_adj, list_logP = data

    list_feature = list_feature.cuda().float()
    list_adj = list_adj.cuda().float()
    list_logP = list_logP.cuda().float().view(-1, 1)

    outputs = net(list_feature, list_adj)

    loss = criterion(outputs, list_logP)        # mse loss

    train_loss += loss.item()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(train_loader)

  return net, train_loss


In [ ]:

def validation(net, partition, optimizer, criterion, args):
  val_loader = torch.utils.data.DataLoader(partition['val'],
                                             batch_size = args.test_batch_size,
                                             shuffle = True,
                                             num_workers = 2
                                             )
  net.eval()

  val_loss = 0.0
  with torch.no_grad():
    for data in val_loader:
      list_feature, list_adj, list_logP = data

      list_feature = list_feature.cuda().float()
      list_adj = list_adj.cuda().float()
      list_logP = list_logP.cuda().float().view(-1, 1)

      outputs = net(list_feature, list_adj)

      loss = criterion(outputs, list_logP)      # mse loss

      val_loss += loss.item()

    val_loss = val_loss / len(val_loader)

  return val_loss


In [ ]:
# 값을 예측하는 모델이기 때문에 accuracy가 아닌 mae, mean std를 평가 지표로 활용한다.

def test(net, partition, args):
  test_loader = torch.utils.data.DataLoader(partition['test'],
                                            batch_size = args.test_batch_size,
                                            shuffle = False, num_workers = 2
                                            )

  net.eval()
  with torch.no_grad():
    
    logP_total = []
    pred_logP_total = []
    for data in test_loader:
      list_feature, list_adj, list_logP = data

      list_feature = list_feature.cuda().float()
      list_adj = list_adj.cuda().float()
      list_logP = list_logP.cuda().float()

      logP_total += list_logP.tolist()
      list_logP = list_logP.view(-1, 1)

      outputs = net(list_feature, list_adj)
      pred_logP_total += outputs.view(-1).tolist()

    mae = mean_absolute_error(logP_total, pred_logP_total)
    std = np.std(np.array(logP_total) - np.array(pred_logP_total))

  return mae, std, logP_total, pred_logP_total


In [ ]:

def experiment(partition, args):
  
  net = GCNNet(args)
  net.cuda()

  criterion = nn.MSELoss()
  optimizer = optim.Adam(net.parameters(), lr = args.lr)

  train_losses = []
  val_losses = []

  for epoch in range(args.EPOCH):
    t_start = time.time()

    net, train_loss = train(net, partition, optimizer, criterion, args)
    val_loss = validation(net, partition, criterion, args)

    t_end = time.time()

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print('epoch : {}, loss (train/val) : {:.4f}/{:.4f}, time : {.2f} sec'.format(epoch, train_loss, val_loss, t_end - t_start))

  mae, std, _, _ = test(net, partition, args)

  result = {}
  result['train_losses'] = train_losses
  result['val_losses'] = val_losses
  result['mae'] = mae
  result['std'] = std
  
  return vars(args), result


